FinalW2vSvmCrossVal.py

In [0]:
# import sys
# sys.path.append('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger')

In [0]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger

/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger


In [0]:
import W2VVectorizer
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from prettytable import PrettyTable
from sklearn.svm import SVC

# Implementation

In [0]:
def main():
    start_time = time.time()
    run_cross_val()
    # run_holdout()
    end_time = time.time()
    print("Time taken for the process: " + str(end_time - start_time))
    return

In [0]:
def run_cross_val():
    all_predictions = []
    w2v_model_path = "/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/word_embedding/word2vec/old/"
    comments = pd.read_csv("/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/corpus/analyzed/comments_tagged_remove_all_punc_keep_question.csv", ";")
    pretty_table = PrettyTable(["Algorithm", "Accuracy", "Precision", "Recall", "F1_Score"])

    i = 1
    kf = KFold(n_splits=10)
    kf.get_n_splits(comments)
    for train_index, test_index in kf.split(comments):
        train_data_comments, test_data_comments = comments["comment"][train_index], comments["comment"][test_index]
        train_data_labels, test_data_labels = comments["label"][train_index], comments["label"][test_index]
        # vectorizer = W2VVectorizer.W2VVectorizer(w2v_model_path + "word2vec_model_skipgram_300", False)
        vectorizer = W2VVectorizer.W2VVectorizer(w2v_model_path + "word2vec_model_skipgram_remove300_10", False)
        predictions = fit_models(vectorizer, train_data_comments, test_data_comments, train_data_labels, test_data_labels)
        all_predictions = all_predictions + predictions.tolist()

        i = i + 1
        evaluation_metrics(test_data_labels, predictions, pretty_table, "iteration" + str(i))

    evaluation_metrics(comments["label"], all_predictions, pretty_table, "final")
    print(pretty_table)
    print_confusion_matrix(comments["label"], all_predictions)

In [0]:
def fit_models(vectorizer, train_data_comments, test_data_comments, train_data_labels, test_data_labels):
    pretty_table = PrettyTable(["Algorithm", "Accuracy", "Precision", "Recall", "F1_Score"])

    vectorized_train_comments = vectorizer.fit_transform(train_data_comments)
    vectorized_test_comments = vectorizer.transform(test_data_comments)

    model = SVC(C=1, kernel='linear')
    model = model.fit(vectorized_train_comments, train_data_labels)
    predictions = model.predict(vectorized_test_comments)
    evaluation_metrics(test_data_labels, predictions, pretty_table, "SVM")
    print_confusion_matrix(test_data_labels, predictions)

    print(pretty_table)
    print("")
    return predictions

In [0]:
def evaluation_metrics(true_sentiment, predicted_sentiment, pretty_table, algorithm):
    label_binarizer = preprocessing.LabelBinarizer()
    label_binarizer.fit(['NEGATIVE', 'POSITIVE'])
    test_labels = label_binarizer.transform(true_sentiment)
    predict_labels = label_binarizer.transform(predicted_sentiment)
    accuracy_str = str(accuracy_score(true_sentiment, predicted_sentiment))
    precision_str = str(precision_score(test_labels, predict_labels))
    recall_str = str(recall_score(test_labels, predict_labels))
    f1_score_str = str(f1_score(test_labels, predict_labels))
    pretty_table.add_row([algorithm, accuracy_str, precision_str, recall_str, f1_score_str])
    return

In [0]:
def print_confusion_matrix(label, prediction):
    cf_matrix = confusion_matrix(label, prediction)
    print(cf_matrix.ravel())
    print(cf_matrix)


In [0]:
main()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[246  24  71 160]
[[246  24]
 [ 71 160]]
+-----------+-------------------+--------------------+--------------------+--------------------+
| Algorithm |      Accuracy     |     Precision      |       Recall       |      F1_Score      |
+-----------+-------------------+--------------------+--------------------+--------------------+
|    SVM    | 0.810379241516966 | 0.8695652173913043 | 0.6926406926406926 | 0.7710843373493975 |
+-----------+-------------------+--------------------+--------------------+--------------------+



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[225  13  51 212]
[[225  13]
 [ 51 212]]
+-----------+-------------------+--------------------+--------------------+--------------------+
| Algorithm |      Accuracy     |     Precision      |       Recall       |      F1_Score      |
+-----------+-------------------+--------------------+--------------------+--------------------+
|    SVM    | 0.872255489021956 | 0.9422222222222222 | 0.8060836501901141 | 0.8688524590163934 |
+-----------+-------------------+--------------------+--------------------+--------------------+



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


...

[202  10  51 238]
[[202  10]
 [ 51 238]]
+-----------+--------------------+--------------------+--------------------+--------------------+
| Algorithm |      Accuracy      |     Precision      |       Recall       |      F1_Score      |
+-----------+--------------------+--------------------+--------------------+--------------------+
|    SVM    | 0.8782435129740519 | 0.9596774193548387 | 0.8235294117647058 | 0.8864059590316573 |
+-----------+--------------------+--------------------+--------------------+--------------------+



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[288  15  40 158]
[[288  15]
 [ 40 158]]
+-----------+--------------------+--------------------+-------------------+--------------------+
| Algorithm |      Accuracy      |     Precision      |       Recall      |      F1_Score      |
+-----------+--------------------+--------------------+-------------------+--------------------+
|    SVM    | 0.8902195608782435 | 0.9132947976878613 | 0.797979797979798 | 0.8517520215633424 |
+-----------+--------------------+--------------------+-------------------+--------------------+



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[321  25  38 117]
[[321  25]
 [ 38 117]]
+-----------+-------------------+-------------------+--------------------+-------------------+
| Algorithm |      Accuracy     |     Precision     |       Recall       |      F1_Score     |
+-----------+-------------------+-------------------+--------------------+-------------------+
|    SVM    | 0.874251497005988 | 0.823943661971831 | 0.7548387096774194 | 0.787878787878788 |
+-----------+-------------------+-------------------+--------------------+-------------------+

+-------------+--------------------+--------------------+--------------------+--------------------+
|  Algorithm  |      Accuracy      |     Precision      |       Recall       |      F1_Score      |
+-------------+--------------------+--------------------+--------------------+--------------------+
|  iteration2 | 0.810379241516966  | 0.8695652173913043 | 0.6926406926406926 | 0.7710843373493975 |
|  iteration3 | 0.872255489021956  | 0.9422222222222222 | 0.8060836501901141 | 0.86